**Problem: Custom Dictionary Queries**

Date: 9/26/2025

You are developing a new programming language. You believe that ordinary dictionaries are boring, so you've decided to add a cool feature to make your language unique!

You want the cool feature to be able to perform two types of queries. With two integer arrays, **a** and **b**, the two types of queries are as follows:

- **If the query is of the form [0, i, x]:** Add `x` to `a[i]` (i.e., `a[i]` should be assigned the value of `a[i] + x`).
- **If the query is of the form [1, x]:** Find the total number of pairs of indices `i` and `j` such that `a[i] + b[j] = x`.

You will be given the arrays of integers **a** and **b**, as well as **queries**, an array of queries in either of the forms described above. Your task is to implement this cool feature, perform the given queries and return an array of the results of the queries of the type **[1, x]**.

---

**Example 1:**

For `a = [1, 4]`, `b = [1, 2, 3]`, and `queries = [[1, 5], [0, 0, 2], [1, 5]]`, the output should be `solution(a, b, queries) = [1, 2]`.

**Trace:**
- Initially: `a = [1, 4]`, `b = [1, 2, 3]`
- Query `[1, 5]`: Find pairs where `a[i] + b[j] = 5`
  - Only one way: `4 + 1 = a[1] + b[0]` → Result: **1**
- Query `[0, 0, 2]`: Add 2 to `a[0]` → `a = [3, 4]`
- Query `[1, 5]`: Find pairs where `a[i] + b[j] = 5`
  - Two ways: `3 + 2 = a[0] + b[1]` and `4 + 1 = a[1] + b[0]` → Result: **2**
- Return: **[1, 2]**

---

**Example 2:**

For `a = [2, 3]`, `b = [1, 2, 2]`, and `queries = [[1, 4], [0, 0, 1], [1, 5]]`, the output should be `solution(a, b, queries) = [3, 4]`.

**Trace:**
- Initially: `a = [2, 3]`, `b = [1, 2, 2]`
- Query `[1, 4]`: Three ways to form 4: `2+2` (twice) and `3+1` → Result: **3**
- Query `[0, 0, 1]`: Add 1 to `a[0]` → `a = [3, 3]`
- Query `[1, 5]`: Four ways to form 5: `3+2` (from both a[0] and a[1] with b[1] and b[2]) → Result: **4**
- Return: **[3, 4]**

---

**Constraints:**
- Execution time limit: 4 seconds (py3)
- Memory limit: 1 GB
- `1 ≤ a.length ≤ 5 × 10^4`
- `0 ≤ a[i] ≤ 10^8`

In [ ]:
from collections import Counter
from bisect import bisect_left, insort

def solution(a, b, queries):
    """
    Handle two types of queries:
    - [0, i, x]: Add x to a[i]
    - [1, x]: Count pairs where a[i] + b[j] = x
    """
    # Frequency maps for values in arrays a and b
    freq_a = Counter(a)
    freq_b = Counter(b)
    
    # Sorted lists of unique values (for efficient two-pointer counting)
    unique_a = sorted(freq_a.keys())
    unique_b = sorted(freq_b.keys())
    
    results = []
    
    for query in queries:
        if query[0] == 0:
            # Update query: add query[2] to a[query[1]]
            index = query[1]
            delta = query[2]
            
            old_value = a[index]
            new_value = old_value + delta
            a[index] = new_value
            
            # Update frequency map and sorted unique values
            freq_a[old_value] -= 1
            if freq_a[old_value] == 0:
                # Old value no longer exists, remove from tracking
                del freq_a[old_value]
                position = bisect_left(unique_a, old_value)
                unique_a.pop(position)
            
            # Add new value to tracking
            if new_value not in freq_a or freq_a[new_value] == 0:
                # New value didn't exist before, add to sorted list
                insort(unique_a, new_value)
            freq_a[new_value] += 1
            
        else:
            # Count query: find pairs where a[i] + b[j] = target
            target = query[1]
            
            # Two-pointer approach on sorted unique values
            left = 0
            right = len(unique_b) - 1
            count = 0
            
            while left < len(unique_a) and right >= 0:
                current_sum = unique_a[left] + unique_b[right]
                
                if current_sum == target:
                    # Found matching pair - multiply frequencies
                    count += freq_a[unique_a[left]] * freq_b[unique_b[right]]
                    left += 1
                    right -= 1
                elif current_sum < target:
                    # Sum too small, need larger value from a
                    left += 1
                else:
                    # Sum too large, need smaller value from b
                    right -= 1
            
            results.append(count)
    
    return results